In [1]:
%cd ../src

/Users/shanekercheval/repos/sik-llms/src


# Clients

In [2]:
# For "registered" clients (via `@Client.register`), the client
# can be created with `create_client` by passing in the model name.
from sik_llms import create_client

client = create_client(
    model_name='gpt-4o-mini',
    temperature=0.1,
)
client

In [3]:
# Or, the client can be directly instantiated
from sik_llms import OpenAI
client = OpenAI(
    model_name='gpt-4o-mini',
    temperature=0.1,
)
client

In [4]:
# Or, the client can be directly instantiated
from sik_llms import Anthropic
client = Anthropic(
    model_name='claude-3-7-sonnet-latest',
    temperature=0.1,
)
client

# Chat

In [5]:
from sik_llms import create_client, user_message, TextChunkEvent

client = create_client(
    model_name='gpt-4o-mini',
    temperature=0.1,
)

message = user_message("What is the capital of France?")
message

{'role': 'user', 'content': 'What is the capital of France?'}

### Run Synchronously

In [6]:
client(messages=[message])

ResponseSummary(input_tokens=14, output_tokens=7, input_cost=2.1e-06, output_cost=4.2e-06, duration_seconds=0.6609218120574951, response='The capital of France is Paris.')

### Stream Asynchronously

In [7]:
responses = []
async for response in client.run_async(messages=[message]):
    if isinstance(response, TextChunkEvent):
        print(response.content, end="")
        responses.append(response)

The capital of France is Paris.

In [8]:
print(response)

input_tokens=14 output_tokens=7 input_cost=2.1e-06 output_cost=4.2e-06 duration_seconds=0.30423903465270996 response='The capital of France is Paris.'


### Claude 

In [9]:
client = create_client(
    model_name='claude-3-7-sonnet-latest',
    temperature=0.1,
)
response = client(messages=[user_message("What is the capital of France?")])
response

ResponseSummary(input_tokens=14, output_tokens=50, input_cost=4.2000000000000004e-05, output_cost=0.00075, duration_seconds=1.310349941253662, response='The capital of France is Paris. Paris is not only the capital but also the largest city in France, known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral.')

# OpenAI Functions/Tools

In [10]:
from sik_llms import Tool, Parameter, RegisteredClients

weather_tool = Tool(
    name='get_weather',
    description="Get the weather for a location.",
    parameters=[
        Parameter(
            name='location',
            type='string',
            required=True,
            description='The city and country for weather info.',
        ),
    ],
)

client = create_client(
    client_type=RegisteredClients.OPENAI_TOOLS,
    model_name='gpt-4o-mini',
    tools=[weather_tool],
)

message = user_message("What is the weather in Paris?")
response = await client.run_async(messages=[message])
# or `response = client(messages=[message])` for synchronous execution
print(response)
print('---')
print(response.tool_prediction)

input_tokens=60 output_tokens=17 input_cost=9e-06 output_cost=1.0199999999999999e-05 duration_seconds=0.5924029350280762 tool_prediction=ToolPrediction(name='get_weather', arguments={'location': 'Paris, France'}, call_id='call_4VhFoGud6HLi2HhIKQQtq4B5') message=None
---
name='get_weather' arguments={'location': 'Paris, France'} call_id='call_4VhFoGud6HLi2HhIKQQtq4B5'


---

# Claude Functions/Tools

In [11]:
from sik_llms import (
    create_client,
    user_message,
    Tool,
    Parameter,
    RegisteredClients,
)

weather_tool = Tool(
    name='get_weather',
    description="Get the weather for a location.",
    parameters=[
        Parameter(
            name='location',
            type='string',
            required=True,
            description='The city and country for weather info.',
        ),
    ],
)

client = create_client(
    client_type=RegisteredClients.ANTHROPIC_TOOLS,
    model_name='claude-3-7-sonnet-latest',
    tools=[weather_tool],
)

message = user_message("What is the weather in Paris?")
response = await client.run_async(messages=[message])
# or `response = client(messages=[message])` for synchronous execution
print(response)
print('---')
print(response.tool_prediction)

input_tokens=394 output_tokens=40 input_cost=0.001182 output_cost=0.0006000000000000001 duration_seconds=1.670184850692749 tool_prediction=ToolPrediction(name='get_weather', arguments={'location': 'Paris, France'}, call_id='toolu_01QRGTT9RmTcx9dzTtdzKAwJ') message=None
---
name='get_weather' arguments={'location': 'Paris, France'} call_id='toolu_01QRGTT9RmTcx9dzTtdzKAwJ'


---

# Structured Outputs via OpenAI

In [12]:
from pydantic import BaseModel
from sik_llms import create_client, system_message, user_message


class CalendarEvent(BaseModel):  # noqa: D101
    name: str
    date: str
    participants: list[str]

client = create_client(
    model_name='gpt-4o-mini',
    response_format=CalendarEvent,
)
messages=[
    system_message("Extract the event information."),
    user_message("Alice and Bob are going to a science fair on Friday."),
]
response = client(messages=messages)
print(response)
print('---')
print(response.response.parsed)

input_tokens=92 output_tokens=18 input_cost=1.38e-05 output_cost=1.08e-05 duration_seconds=0.7497479915618896 response=StructuredOutputResponse(parsed=CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob']), refusal=None)
---
name='Science Fair' date='Friday' participants=['Alice', 'Bob']


---

# Structured Outputs via Anthropic

In [13]:
from pydantic import BaseModel
from sik_llms import create_client, system_message, user_message


class CalendarEvent(BaseModel):  # noqa: D101
    name: str
    date: str
    participants: list[str]

client = create_client(
    model_name='claude-3-7-sonnet-latest',
    response_format=CalendarEvent,
)
messages=[
    system_message("Extract the event information."),
    user_message("Alice and Bob are going to a science fair on Friday."),
]
response = client(messages=messages)
print(response)
print('---')
print(response.response.parsed)

input_tokens=458 output_tokens=78 input_cost=0.001374 output_cost=0.00117 duration_seconds=1.7810969352722168 response=StructuredOutputResponse(parsed=CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob']), refusal=None)
---
name='Science Fair' date='Friday' participants=['Alice', 'Bob']


---

# Reasoning via OpenAI

In [14]:
from sik_llms import (
    create_client,
    user_message,
    TextChunkEvent,
    ResponseSummary,
    ReasoningEffort,
)

client = create_client(
    model_name='o3-mini',
    reasoning_effort=ReasoningEffort.MEDIUM,
)
messages=[user_message("What is 1 + 2 + (3 * 4) + (5 * 6)?")]
summary = None
async for response in client.run_async(messages=messages):
    if isinstance(response, TextChunkEvent):
        print(response.content, end="")
    elif isinstance(response, ResponseSummary):
        summary = response
    else:
        raise ValueError(f"Unexpected response type: {response}")

Let's break it down step by step:

1. First, calculate the multiplications:
   - 3 * 4 = 12
   - 5 * 6 = 30

2. Now, substitute those back into the expression:
   - 1 + 2 + 12 + 30

3. Finally, add them together:
   - 1 + 2 = 3
   - 3 + 12 = 15
   - 15 + 30 = 45

So, the answer is 45.

In [15]:
summary

ResponseSummary(input_tokens=28, output_tokens=113, input_cost=3.08e-05, output_cost=0.0004972, duration_seconds=2.5976948738098145, response="Let's break it down step by step:\n\n1. First, calculate the multiplications:\n   - 3 * 4 = 12\n   - 5 * 6 = 30\n\n2. Now, substitute those back into the expression:\n   - 1 + 2 + 12 + 30\n\n3. Finally, add them together:\n   - 1 + 2 = 3\n   - 3 + 12 = 15\n   - 15 + 30 = 45\n\nSo, the answer is 45.")

---

# Reasoning via Claude

In [16]:
from sik_llms import (
    create_client,
    user_message,
    TextChunkEvent,
    ThinkingChunkEvent,
    ResponseSummary,
    ReasoningEffort,
)

client = create_client(
    model_name='claude-3-7-sonnet-latest',
    reasoning_effort=ReasoningEffort.MEDIUM,
)
messages=[user_message("What is 1 + 2 + (3 * 4) + (5 * 6)?")]
summary = None

current_type = None
async for response in client.run_async(messages=messages):
    is_text_chunk = isinstance(response, TextChunkEvent)
    is_thinking_chunk = isinstance(response, ThinkingChunkEvent)
    is_summary = isinstance(response, ResponseSummary)

    if is_text_chunk or is_thinking_chunk:
        if type(response) is not current_type:
            print(f"\n\n[{'THINKING' if is_thinking_chunk else 'TEXT'}]")
            current_type = type(response)
        print(response.content, end="")
    elif isinstance(response, ResponseSummary):
        summary = response
    else:
        raise ValueError(f"Unexpected response type: {response}")



[THINKING]
I need to calculate 1 + 2 + (3 * 4) + (5 * 6).

Let me break it down step by step, following the order of operations (PEMDAS: Parentheses, Exponents, Multiplication/Division, Addition/Subtraction).

First, let's calculate the expressions in parentheses:
- (3 * 4) = 12
- (5 * 6) = 30

Now, our expression becomes: 1 + 2 + 12 + 30

Adding these numbers:
1 + 2 = 3
3 + 12 = 15
15 + 30 = 45

So, 1 + 2 + (3 * 4) + (5 * 6) = 45.

[TEXT]
To solve this expression, I'll follow the order of operations:

1) First, calculate the expressions in parentheses:
   - (3 * 4) = 12
   - (5 * 6) = 30

2) Now add all terms together:
   1 + 2 + 12 + 30 = 45

The answer is 45.

---

# ReasoningAgent

In [5]:
import json
from sik_llms.models_base import (
    Tool, Parameter, ThinkingEvent, ToolPredictionEvent, 
    ToolResultEvent, TextChunkEvent, ErrorEvent, ResponseSummary
)
from sik_llms.reasoning_agent import ReasoningAgent

# Define a calculator tool
calculator_tool = Tool(
    name="calculator",
    description="Perform mathematical calculations",
    parameters=[
        Parameter(
            name="expression", 
            type="string", 
            required=True, 
            description="The mathematical expression to evaluate (e.g., '2 + 2', '5 * 10')"
        )
    ]
)

# Define a weather tool
weather_tool = Tool(
    name="get_weather",
    description="Get the current weather for a location",
    parameters=[
        Parameter(
            name="location", 
            type="string", 
            required=True, 
            description="The name of the city(e.g., 'San Francisco', 'New York', 'London')",
        )
    ]
)

# Define the tool executors
async def calculator_executor(expression: str) -> str:
    """Execute calculator tool."""
    try:
        # Only allow simple arithmetic for safety
        allowed_chars = set("0123456789+-*/() .")
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
        return str(eval(expression))
    except Exception as e:
        return f"Error: {str(e)}"

async def weather_executor(location: str) -> str:
    """Mock weather tool - returns fake data."""
    # Return mock weather data
    weather_data = {
        "New York": "68°F, Partly Cloudy",
        "San Francisco": "62°F, Foggy",
        "Miami": "85°F, Sunny",
        "Chicago": "55°F, Windy",
        "Los Angeles": "75°F, Clear"
    }

    for city in weather_data:  # noqa: PLC0206
        if city.lower() in location.lower():
            return f"Weather for {location}: {weather_data[city]}"

    return f"Weather for {location} not found"

tool_executors = {
    "calculator": calculator_executor,
    "get_weather": weather_executor
}

# Create the reasoning agent
agent = ReasoningAgent(
    model_name="gpt-4o-mini",  # You can change this to other models
    tools=[calculator_tool, weather_tool],
    tool_executors=tool_executors,
    max_iterations=10,
    temperature=0,
)

question = "I'm planning a trip to New York and Miami. What's the weather like in both cities? Also, if I have a budget of $2400 for a 6-day trip, how much can I spend per day?"
# Run the agent and collect the results
messages = [{"role": "user", "content": question}]

print(f"[QUESTION]: {question}\n")

current_iteration = 0

async for event in agent.run_async(messages):
    if isinstance(event, ThinkingEvent):
        if hasattr(event, 'iteration') and event.iteration != current_iteration:
            current_iteration = event.iteration
            print(f"\n--- Iteration {current_iteration}\n")
        if event.content:
            print(f"\n[THINKING]:\n{event.content}")

    elif isinstance(event, ToolPredictionEvent):
        print("\n[TOOL PREDICTION]:")
        print(f"Tool: {event.name}`")
        print(f"Parameters: \n```json\n{json.dumps(event.arguments, indent=2)}\n```")

    elif isinstance(event, ToolResultEvent):
        print("\n[TOOL RESULT]:")
        print(f"Tool: {event.name}`")
        print(f"Result: {event.result}")

    elif isinstance(event, ErrorEvent):
        print("\n[ERROR]:")
        print(f"Error: {event.content}")

    elif isinstance(event, TextChunkEvent):
        # For streaming text generation
        if current_iteration  >= 0:  # Only print once for the header
            print("\n[FINAL RESPONSE]:")
            current_iteration = -1  # Set to an impossible iteration to avoid repeating

        print(event.content, end="")

    elif isinstance(event, ResponseSummary):
        # Print nothing here as we've already streamed the response
        pass

# Just for notebook display cleanliness
print("\n\n---\n\n")

[QUESTION]: I'm planning a trip to New York and Miami. What's the weather like in both cities? Also, if I have a budget of $2400 for a 6-day trip, how much can I spend per day?


--- Iteration 1


[THINKING]:
I need to gather the current weather information for both New York and Miami. After that, I will calculate the daily budget for the trip based on the total budget of $2400 for 6 days.

[TOOL PREDICTION]:
Tool: get_weather`
Parameters: 
```json
{
  "location": "New York"
}
```

[TOOL RESULT]:
Tool: get_weather`
Result: Weather for New York: 68°F, Partly Cloudy

--- Iteration 2


[THINKING]:
I have obtained the current weather for New York, which is 68°F and partly cloudy. Now, I will gather the weather information for Miami to complete the weather comparison.

[TOOL PREDICTION]:
Tool: get_weather`
Parameters: 
```json
{
  "location": "Miami"
}
```

[TOOL RESULT]:
Tool: get_weather`
Result: Weather for Miami: 85°F, Sunny

--- Iteration 3


[THINKING]:
I have now gathered the weather

In [18]:
print(f"Input Tokens: {event.input_tokens}")
print(f"Output Tokens: {event.output_tokens}")
print(f"Total Cost: {event.total_cost}")
print(f"Duration: {event.duration_seconds:.2f} seconds")

Input Tokens: 6299
Output Tokens: 493
Total Cost: 0.0012406499999999998
Duration: 14.24 seconds
